**After this competition, I think I like recommendation, But I didn't do better. I didn't use ranker in this competition because I don't have any experience in this area, users and items characteristics? Label 1 or 0? training data? what is worse is that when the competition is about to end, I found out that I needed to use ranker to do better.**

**Here I would provide my ideas for recommending, and thank them for giving me ideas, although I may not be familiar with those, Finally I hoping someone can provide code to let me know How ranker/classifer works after the competition. Anyway, I really enjoyed the game and I learned a lot.**

In [ ]:
import pandas as pd
import numpy as np
import warnings
import gc
from tqdm import tqdm
from datetime import datetime

In [ ]:
warnings.filterwarnings('ignore')
tqdm.pandas()

# Do customers buy items they bought before?
The source of this idea is these Notebook:
1. https://www.kaggle.com/code/hervind/h-m-faster-trending-products-weekly/notebook
2. https://www.kaggle.com/code/lichtlab/do-customers-buy-the-same-products-again

In [ ]:
%%time
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv', 
                                 usecols = ['t_dat', 'customer_id', 'article_id'], dtype={'article_id': str})
transactions_train['t_dat'] = pd.to_datetime(transactions_train['t_dat'])
lastday = transactions_train['t_dat'].max()
transactions_train

In [ ]:
%%time
transactions_train['dow'] = transactions_train['t_dat'].dt.dayofweek
transactions_train['ldbw'] = transactions_train['t_dat'] - pd.TimedeltaIndex(transactions_train['dow'] - 1, unit='D')
transactions_train.loc[transactions_train['dow'] >=2 , 'ldbw'] = \
                transactions_train.loc[transactions_train['dow'] >=2 , 'ldbw'] + \
                pd.TimedeltaIndex(np.ones(len(transactions_train.loc[transactions_train['dow'] >=2])) * 7, unit='D')
del transactions_train['dow']
transactions_train

In [ ]:
%%time
weekly_sales = transactions_train.drop(['customer_id'], axis=1).groupby(['ldbw', 'article_id']).count()
weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})
transactions_train = pd.merge(transactions_train, weekly_sales, on=['ldbw', 'article_id'], how='left')
transactions_train

In [ ]:
%%time
weekly_sales = weekly_sales.reset_index().set_index('article_id')
last_day = lastday.strftime('%Y-%m-%d')

transactions_train = transactions_train.join(
    weekly_sales.loc[weekly_sales['ldbw']==last_day, ['count']],
    on='article_id', rsuffix="_targ")

transactions_train['count_targ'].fillna(0, inplace=True)
# The count_targ column is to check the number of products sold in the past week, 
# and NAN means that the product has not been sold in this week
transactions_train

**quotient** can be understood as the popularity of hot-selling products of the last week in each week

In [ ]:
transactions_train['quotient'] = transactions_train['count_targ'] / transactions_train['count']
transactions_train

**value** can be understood as the consumer's liking for a certain product, which includes many factors (purchase time, number of purchases, and the popularity of the product (quotient))

In [ ]:
%%time
train_fo = transactions_train.copy()
train_fo['week_x'] = ((lastday - train_fo['ldbw']) / np.timedelta64(7, 'D')).astype(int)
train_fo['sup_1'] = 0.1 
train_fo['week_x'] = train_fo[['week_x', 'sup_1']].max(axis=1)

train_fo['y'] = 1.2e3 / train_fo['week_x']
train_fo['sup_25'] = 25 
train_fo['y'] = train_fo[["y", "sup_25"]].max(axis=1)
train_fo['value'] = train_fo['quotient'] * train_fo['y'] 

value = train_fo.groupby(['customer_id', 'article_id']).agg({'value': 'sum'})
value = value.reset_index()
value

In [ ]:
%%time
train_fo = train_fo.groupby(['customer_id', 'article_id']).agg({'value': 'sum'})
train_fo = train_fo.reset_index()

train_fo = train_fo.loc[train_fo['value'] > 180]
# why 180? If an item's quotient remains at 1, users will buy it again if it has been purchased about the last 7 weeks
train_fo['rank'] = train_fo.groupby("customer_id")["value"].rank("dense", ascending=False)
train_fo = train_fo.loc[train_fo['rank'] <= 12]

purchase_records = train_fo.sort_values(['customer_id', 'value'], ascending = False).reset_index(drop = True)
purchase_records['prediction'] = purchase_records['article_id'].astype(str) + ' '
purchase_records = purchase_records.groupby('customer_id').agg({'prediction': sum}).reset_index() #str求和
purchase_records['prediction'] = purchase_records['prediction'].str.strip()
purchase_records

**pairing these items**

the idea from the notebook: https://www.kaggle.com/code/titericz/article-id-pairs-in-3s-using-cudf

In [ ]:
def calc_pairs(train):
    # Calculate all articles purchased together
    dt = train.groupby(['customer_id','t_dat'])['article_id'].agg(list).rename('pair').reset_index()
    df = pd.merge(train[['customer_id', 't_dat', 'article_id']], dt, on=['customer_id', 't_dat'], how='left')
    del dt
    gc.collect()

    # baes_count
    base_count = df.groupby(['article_id']).size().rename('base_count').reset_index()
    gc.collect()
    
    # Explode the rows vs list of articles
    df = df[['article_id', 'pair']].explode(column='pair')
    gc.collect()
    
    # Discard duplicates
    df = df.loc[df['article_id']!=df['pair']].reset_index(drop=True)
    gc.collect()

    # Count how many times each pair combination happens
    df = df.groupby(['article_id', 'pair']).size().rename('pair_count').reset_index()
    gc.collect()
    
    # Sort by frequency
    df = df.sort_values(['article_id' ,'pair_count'], ascending=[False, False]).reset_index(drop=True)
    df = pd.merge(df, base_count, on=['article_id'], how='left')
    df['ratio'] = df['pair_count'] / df['base_count']
    gc.collect()
    
    # Pick only top1 most frequent pair
    df = df.groupby('article_id').head(2)
    gc.collect()
    
    return df

In [ ]:
%%time
articles_pair = calc_pairs(transactions_train[transactions_train['t_dat']>datetime(2020,6,22)])
articles_pair

In [ ]:
articles_pair_base = articles_pair[articles_pair['ratio']>0.12]
articles_pair_base.sort_values('ratio', ascending=False, inplace=True)
articles_pair_base = articles_pair_base.groupby('article_id')['pair'].agg(list).reset_index()
del articles_pair
articles_pair_base

In [ ]:
def pair_zh(train, pair_train):
    
    train = train.explode('prediction')
    train = pd.merge(train, pair_train, left_on='prediction', right_on='article_id', how='left')
    train.fillna('0', inplace=True)
    for col in ['article_id']:
        del train[col]
        
    def flat(x):
        res = []
        if isinstance(x, str):
            res.append(x)
        else:
            res.extend(x)
        unique_list = []
        for i in res:
            if i not in unique_list and i != '0':
                unique_list.append(i)
        re = []
        for i in unique_list:
            if isinstance(i, list):
                re.extend(i)
            else:
                re.append(i)
        return re
    prediction = train.groupby('customer_id')['prediction'].apply(lambda x:flat(x))
    pair = train.groupby('customer_id')['pair'].progress_apply(lambda x:flat(x))
    train['prediction'] = train['customer_id'].map(prediction)
    train['pair'] = train['customer_id'].map(pair)
    train.drop_duplicates('customer_id', inplace=True)
    
    def falt(x):
        res = []
        for i in x:
            res.extend(i)
        unique_list = []
        for i in res:
            if i not in unique_list:
                unique_list.append(i)
        return unique_list[:12]
    train['prediction'] = train[['prediction', 'pair']].apply(lambda x:falt(x.tolist()), axis='columns')
    del train['pair']
    
    return train

In [ ]:
purchase_records['prediction'] = purchase_records['prediction'].apply(lambda x:x.split(' '))
purchase_records = pair_zh(purchase_records, articles_pair_base)
del articles_pair_base
gc.collect()
purchase_records

These purchased products and pairs are the key to a not bad score, and the subsequent recommendations have far less improvement in the final score than the above recommendations.

**Cut consumers by age and find the most popular items in the last week**

In [ ]:
%%time
customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv', 
                        usecols=['customer_id', 'age'])
bins = list(range(16,81,4))+ [100]
names = list(range(len(bins)-1))
customers['age_group'] = pd.cut(customers['age'], bins, labels=names, right=False)
del customers['age']
customers['age_group'] = customers['age_group'].astype(float).fillna(99).astype(int)


agegroup = transactions_train[['t_dat', 'customer_id', 'article_id']]
agegroup = agegroup[agegroup['t_dat'] > datetime(2020,9,15)][['customer_id', 'article_id']]
agegroup = pd.merge(agegroup, customers, on='customer_id', how='left')
del agegroup['customer_id']
agegroup = agegroup[agegroup['age_group'] != 99]

def generate_age_top(agegroup, num11, num22):
    
    def hope(x, num1, num2):
        return ' '.join(x.value_counts().index.tolist()[num1:num2])
    age_articles = agegroup[['age_group', 'article_id']].groupby('age_group')['article_id'].\
                   progress_apply(lambda x:hope(x, num11, num22)).reset_index()
    age_articles.loc[17,'age_group'] = 99
    age_articles.loc[17,'article_id'] = ' '.join(agegroup['article_id'].value_counts().index.tolist()[num11:num22])
    age_articles['article_id'] = age_articles['article_id'].map(lambda x:x.split(' '))
    
    return age_articles

age_articles = generate_age_top(agegroup,0,12)
age_articles_ = age_articles.copy()
age_articles_['article_id'] = age_articles_['article_id'].apply(lambda x:' '.join(x))
age_articles_

In [ ]:
lastday = transactions_train['t_dat'].max()
transactions_train['purchaseday_to_last'] = lastday - transactions_train['t_dat'] 
transactions_train['purchaseday_to_last'] = transactions_train.groupby(['customer_id', 'article_id'])['purchaseday_to_last'].transform(min)
transactions_train['purchaseday_to_last'] = transactions_train['purchaseday_to_last'].dt.days
for col in ['ldbw', 'count', 'count_targ', 'quotient']:
    del transactions_train[col]
transactions_train

In [ ]:
purchase_times = transactions_train.groupby(['customer_id', 'article_id'])['t_dat'].count().\
                 reset_index().rename(columns={'t_dat':'purchase_times'})
transactions_train_pt = pd.merge(transactions_train, purchase_times, on=['customer_id', 'article_id'], how='left')
transactions_train_pt.drop_duplicates(['customer_id', 'article_id'], keep='last', inplace=True)
transactions_train_pt = pd.merge(transactions_train_pt, value, on=['customer_id', 'article_id'], how='left')
del transactions_train
transactions_train_pt

In [ ]:
%%time
articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv',
                       usecols = ['article_id', 'detail_desc', 'index_group_no'], dtype={'article_id': str})
articles.rename(columns={'detail_desc':'detail_feature_no', 'index_group_no':'population_no'}, inplace=True)
# articles_sales_week = transactions_train[transactions_train['t_dat'] > datetime(2020,9,15)].\
#                       groupby(['article_id'])['t_dat'].count()
# articles_sales_month = transactions_train[transactions_train['t_dat'] > datetime(2020,8,22)].\
#                       groupby(['article_id'])['t_dat'].count()
# del transactions_train
# articles = pd.merge(articles, articles_sales_week, on='article_id', how='left')
# articles.rename(columns={'t_dat':'sales_week'}, inplace=True)
# articles = pd.merge(articles, articles_sales_month, on='article_id', how='left')
# articles.rename(columns={'t_dat':'sales_month'}, inplace=True)
# articles['sales_week'].fillna(0, inplace=True)
# articles['sales_month'].fillna(0, inplace=True)
# articles.sort_values(by=['sales_week', 'sales_month'], ascending=[False, False], inplace=True)
articles['detail_feature_no'] = pd.factorize(articles['detail_feature_no'])[0]
articles

In [ ]:
def articles_table(data):
    # 商品推荐表
    data['label_A'] = data['detail_feature_no'].astype(str)
    data['label_B'] = data['population_no'].astype(str) 
    
    for col in ['detail_feature_no', 'population_no']:
        del data[col]
    return data

articles = articles_table(articles)
articles

In [ ]:
transactions_train_pt = pd.merge(transactions_train_pt, articles, on='article_id', how='left')
gc.collect()
transactions_train_pt

# LABEL RECOMMENDATION
1. use labelA to recommend similar products to the top 5 products with the highest value of each consumer.
2. use one categorie of products labelB with the most purchases from the 12 products with the highest value to recommend

In [ ]:
def recommand(train, articles, label, A_num=5, B_num=60, C_num=12):
    # 提取
    def get_likegroup(x,num):
        return x.tolist()[:num]
    recommend_A_mapgroup = articles.groupby(label)['article_id'].apply(lambda x:get_likegroup(x,A_num))
    
    train['article_id_recommend'] = train[label].map(recommend_A_mapgroup)
    train.fillna('0', inplace=True)
    for col in ['label_A', 'label_B']:
        del train[col]

    def flat(x, lnum):
        res = []
        for i in x:
            res.extend(i)
        unique_list = []
        for i in res:
            if i not in unique_list and i != '0':
                unique_list.append(i)
        return unique_list[:lnum]
    temp = train[['customer_id', 'article_id_recommend']].\
                     groupby('customer_id')['article_id_recommend'].apply(lambda x:flat(x.tolist(), lnum=B_num))
    train['prediction'] = train['customer_id'].map(temp)
    for col in ['article_id', 'purchaseday_to_last', 'purchase_times', 't_dat', 'article_id_recommend', 'value',
                'age_group']:
        del train[col]
    train.drop_duplicates('customer_id', inplace=True)
    train['count'] = train['prediction'].apply(lambda x:len(x))
    train = train[train['count']>0]
    del train['count']  
    
    train = train.explode('prediction')
    train = pd.merge(train, articles[['article_id', 'rank']], left_on='prediction', right_on='article_id', how='left')
    train.sort_values('rank', ascending=True, inplace=True)
    temp = train.groupby('customer_id')['prediction'].apply(lambda x:x.tolist()[:C_num])
    train['prediction'] = train['customer_id'].map(temp)
    for col in ['article_id', 'rank']:
        del train[col]
    train.drop_duplicates('customer_id', inplace=True)

    return train

In [ ]:
def age_rank_rec(train, train_age, customers, articles, label_rec ,rank, num1=5, num2=60, num3=12):    
    
    res = pd.DataFrame()
    train_age = pd.merge(train_age, customers, on='customer_id', how='left')

    for i in tqdm(list(range(len(bins)-1)) + [99]):

        train_age_temp = train_age[train_age['age_group'] == i]
        train_temp = train[train['age_group'] == i]


        articles_sales_week_temp = train_age_temp[train_age_temp['t_dat'] > datetime(2020,9,15)].\
                    groupby(['article_id'])['t_dat'].count().reset_index().rename(columns={'t_dat':'sale_week_temp'})
        articles_sales_month_temp = train_age_temp[train_age_temp['t_dat'] > datetime(2020,8,22)].\
                    groupby(['article_id'])['t_dat'].count().reset_index().rename(columns={'t_dat':'sale_month_temp'})

       
        articles_temp = articles.copy()
        articles_temp = pd.merge(articles_temp, articles_sales_week_temp, on='article_id', how='left')
        articles_temp = pd.merge(articles_temp, articles_sales_month_temp, on='article_id', how='left')
        articles_temp['sale_week_temp'].fillna(0, inplace=True)
        articles_temp['sale_month_temp'].fillna(0, inplace=True)
        articles_temp.sort_values(by=['sale_week_temp', 'sale_month_temp'], inplace=True, ascending=[False, False])
        articles_temp = articles_temp[articles_temp['sale_week_temp']>0]
        articles_temp['rank'] = articles_temp['sale_week_temp'].rank(method='first',ascending=False)
        articles_temp = articles_temp[['article_id', label_rec, 'rank']][articles_temp['rank']<=rank]

        rec = recommand(train_temp, articles_temp, label=label_rec, A_num=num1, B_num=num2, C_num=num3)
        res = pd.concat([res, rec])
        del rec
        gc.collect()

    return res

In [ ]:
train_head5 = transactions_train_pt.copy()
train_head5.sort_values(by=['value', 'purchase_times'], ascending=[False, False], inplace=True)
train_head5 = train_head5.groupby('customer_id').head(5)
train_head5 = pd.merge(train_head5, customers, on='customer_id', how='left')
train_head5

In [ ]:
%%time
rec_A = age_rank_rec(train_head5, transactions_train_pt, customers, 
                     articles, label_rec='label_A', rank=1000, num1=5, num2=20, num3=8)
del train_head5
gc.collect()
rec_A

In [ ]:
train_B_lov1 = transactions_train_pt.copy()
train_B_lov1.sort_values(by=['value', 'purchase_times'], ascending=[False, False], inplace=True)
train_B_lov1 = train_B_lov1.groupby('customer_id').head(6)
train_B_lov1 = pd.merge(train_B_lov1, customers, on='customer_id', how='left')
temp = train_B_lov1.groupby('customer_id')['label_B'].progress_apply(lambda x:x.value_counts().index.tolist()[:1])
train_B_lov1['label_B'] = train_B_lov1['customer_id'].map(temp)
train_B_lov1.drop_duplicates('customer_id', inplace=True)
train_B_lov1 = train_B_lov1.explode('label_B')
train_B_lov1

In [ ]:
%%time
rec_B = age_rank_rec(train_B_lov1, transactions_train_pt, customers, 
                     articles, label_rec='label_B', rank=800, num1=12, num2=24, num3=12)
del train_B_lov1
gc.collect()
rec_B

In [ ]:
sub = customers['customer_id']
sub = pd.merge(sub, purchase_records, on='customer_id', how='left')
sub = pd.merge(sub, rec_A, on='customer_id', suffixes=['_a', '_b'], how='left')
sub = pd.merge(sub, rec_B, on='customer_id', how='left')
sub.fillna('0', inplace=True)
def wash(x):
    res = []
    for i in x:
        res.extend(i)
    unique_list = []
    for i in res:
        if i not in unique_list and i != '0':
            unique_list.append(i)
    return unique_list
sub['prediction'] = sub[['prediction_a', 'prediction_b', 'prediction']].progress_apply(lambda x:wash(x.tolist()), axis=1)
sub['count'] = sub['prediction'].progress_apply(lambda x:len(x))
sub = sub[sub['count'] > 0]
for col in ['prediction_a', 'prediction_b', 'count']:
    del sub[col]
sub

In [ ]:
train_foo = sub.copy()
train_foo['prediction'] = train_foo['prediction'].progress_apply(lambda x:' '.join(x))
train_foo

In [ ]:
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv', usecols=['customer_id'])
no_sub = sub[~sub['customer_id'].isin(train_foo['customer_id'])]
no_sub = pd.merge(no_sub, customers, on='customer_id', how='left')
no_sub = pd.merge(no_sub, age_articles, on='age_group', how='left')
no_sub.rename(columns={'article_id':'prediction'}, inplace=True)
del no_sub['age_group']
no_sub['prediction'] = no_sub['prediction'].map(lambda x:' '.join(x)[:131])
no_sub

In [ ]:
sub = pd.concat([train_foo, no_sub])

In [ ]:
sub = pd.merge(sub, customers, on='customer_id', how='left')
sub = pd.merge(sub, age_articles_, on='age_group', how='left')
del sub['age_group']
sub['prediction'] = sub['prediction'] + ' '+ sub['article_id']
del sub['article_id']
sub['prediction'] = sub['prediction'].map(lambda x:x.lstrip()[:131])
sub

In [ ]:
sub.to_csv('submission.csv', index=False)